In [0]:
from pyspark.sql.functions import col, lit, count, countDistinct, sum, avg, max, date_format, explode

# --- 1. CONFIGURARE UNITY CATALOG ---
CATALOG_NAME = "spotify_etl"
SILVER_SCHEMA = "silver"
GOLD_SCHEMA = "gold"

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{GOLD_SCHEMA}")
print(f"Sursa (Silver): {CATALOG_NAME}.{SILVER_SCHEMA}")
print(f"Destinația (Gold): {CATALOG_NAME}.{GOLD_SCHEMA}")

# --- 2. ÎNCĂRCARE TABELE SILVER ---
fct_plays = spark.table(f"{CATALOG_NAME}.{SILVER_SCHEMA}.fct_plays")
dim_tracks = spark.table(f"{CATALOG_NAME}.{SILVER_SCHEMA}.dim_tracks")
dim_artists = spark.table(f"{CATALOG_NAME}.{SILVER_SCHEMA}.dim_artists")
dim_time = spark.table(f"{CATALOG_NAME}.{SILVER_SCHEMA}.dim_time")

# --- 3. CREAREA UNUI VIEW ANALITIC DE BAZĂ (JOIN) ---
# Acest view combină tabelele silver pentru a facilita agregările
base_analytics_df = fct_plays \
    .join(dim_tracks, "track_id", "left") \
    .join(dim_artists, fct_plays.artist_id == dim_artists.artist_id, "left") \
    .join(dim_time, "play_timestamp", "left") \
    .select(
        fct_plays.play_id,
        fct_plays.play_timestamp,
        fct_plays.duration_ms,
        fct_plays.context_type,
        dim_tracks.track_id,
        dim_tracks.track_name,
        dim_tracks.popularity,
        dim_artists.artist_id,
        dim_artists.artist_name,
        dim_artists.genres,
        dim_time.hour_of_day,
        dim_time.weekday_name,
        date_format(fct_plays.play_timestamp, "yyyy-MM").alias("month")
    )

# Memorăm în cache pentru performanță și creăm un Temp View
base_analytics_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.base_analytics")
print("View-ul analitic 'v_base_analytics' a fost creat și memorat în cache.")

In [0]:
# Deoarece API-ul nu oferă un ID de utilizator distinct, vom agrega totul
# pentru un singur utilizator, "default_user".

print("Creating gold.gold_user_listening_summary...")

# Folosim Spark SQL pe view-ul nostru
gold_user_summary = spark.sql("""
    WITH user_summary AS (
        SELECT
            'default_user' AS user_id,
            COUNT(play_id) AS total_tracks,
            COUNT(DISTINCT artist_id) AS unique_artists,
            SUM(duration_ms / 60000.0) AS total_minutes
        FROM v_base_analytics
    ),
    top_artist AS (
        SELECT
            artist_name AS favorite_artist,
            COUNT(*) AS plays
        FROM v_base_analytics
        GROUP BY artist_name
        ORDER BY plays DESC
        LIMIT 1
    ),
    top_track AS (
        SELECT
            track_name AS top_track,
            COUNT(*) AS plays
        FROM v_base_analytics
        GROUP BY track_name
        ORDER BY plays DESC
        LIMIT 1
    )
    SELECT
        us.user_id,
        us.total_tracks,
        us.unique_artists,
        us.total_minutes,
        ta.favorite_artist,
        tt.top_track
    FROM user_summary us
    CROSS JOIN top_artist ta
    CROSS JOIN top_track tt
""")

gold_user_summary.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_user_listening_summary")
    
print(f"Tabela {CATALOG_NAME}.{GOLD_SCHEMA}.gold_user_listening_summary a fost creată.")

In [0]:
print("Creating gold.gold_top_tracks...")

gold_top_tracks = spark.sql("""
    SELECT
        track_id,
        track_name,
        artist_name,
        COUNT(play_id) AS play_count,
        SUM(duration_ms / 60000.0) AS total_minutes,
        MAX(popularity) AS popularity
    FROM v_base_analytics
    WHERE track_id IS NOT NULL
    GROUP BY track_id, track_name, artist_name
    ORDER BY play_count DESC
""")

gold_top_tracks.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_top_tracks")

print(f"Tabela {CATALOG_NAME}.{GOLD_SCHEMA}.gold_top_tracks a fost creată.")

In [0]:
print("Creating gold.gold_top_artists...")

gold_top_artists = spark.sql("""
    SELECT
        artist_id,
        artist_name,
        COUNT(play_id) AS total_plays,
        COUNT(DISTINCT track_id) AS unique_tracks,
        SUM(duration_ms / 60000.0) AS total_minutes,
        FIRST(genres) as genres -- Preluăm genurile
    FROM v_base_analytics
    WHERE artist_id IS NOT NULL
    GROUP BY artist_id, artist_name
    ORDER BY total_plays DESC
""")

gold_top_artists.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_top_artists")
    
print(f"Tabela {CATALOG_NAME}.{GOLD_SCHEMA}.gold_top_artists a fost creată.")

In [0]:
print("Creating gold.gold_genre_trends...")

# Aici folosim "explode" pentru a transforma array-ul de genuri în rânduri separate
gold_genre_trends = spark.sql("""
    SELECT
        genre,
        month,
        COUNT(play_id) AS total_plays,
        AVG(popularity) AS avg_popularity,
        COUNT(DISTINCT 'default_user') AS unique_users -- Hardcodat
    FROM v_base_analytics
    LATERAL VIEW explode(genres) exploded_genres AS genre
    WHERE genre IS NOT NULL
    GROUP BY genre, month
    ORDER BY month, total_plays DESC
""")

gold_genre_trends.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_genre_trends")
    
print(f"Tabela {CATALOG_NAME}.{GOLD_SCHEMA}.gold_genre_trends a fost creată.")

In [0]:
print("Creating gold.gold_listening_patterns...")

gold_listening_patterns = spark.sql("""
    SELECT
        'default_user' as user_id,
        hour_of_day,
        weekday_name,
        SUM(duration_ms / 60000.0) AS total_minutes
        -- avg_session_length este complex și necesită o analiză a log-urilor, omitem deocamdată
        -- most_common_device este indisponibil din API
    FROM v_base_analytics
    GROUP BY user_id, hour_of_day, weekday_name
    ORDER BY hour_of_day, weekday_name
""")

gold_listening_patterns.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable(f"{CATALOG_NAME}.{GOLD_SCHEMA}.gold_listening_patterns")
    
print(f"Tabela {CATALOG_NAME}.{GOLD_SCHEMA}.gold_listening_patterns a fost creată.")